In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
from time import perf_counter

sys.path.append("/Net/Groups/BSI/work_scratch/ppapastefanou/src/QPy")

from src.postprocessing.qnc_defintions import Time_Reduction_Type
from src.postprocessing.qnc_output_parser import QNC_output_parser
from src.postprocessing.qnc_ncdf_reader import QNC_ncdf_reader
from src.postprocessing.qnc_rescaler import QNC_Rescaler
from src.postprocessing.qnc_obs_reader import QNC_obs_reader

In [2]:
os.listdir("/Net/Groups/BSI/scratch/ppapastefanou/simulations/QPy")

['24_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs_work_high_gammastem',
 '22_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs_work',
 '25_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs_work_high_gammastem',
 '20_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs_work',
 '06_transient_latin_hypercube',
 '15_transient_latin_hypercube_with_std_HAINICH_data_full_2024',
 '23_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs_work_high_gammastem',
 '09_transient_latin_hypercube_with_std',
 '22_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs_work_high_gammastem',
 '19_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs_work',
 '18_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs',
 '06_transient_latin_hypercube_work',
 '13_transient_latin_hypercube_with_std_HAINICH_data_full',
 '14_transient_latin_hypercube_with_std_HAINICH_data_full',
 '16_transient_latin_hypercube_with_std_HAINICH_data_full_2024_

In [22]:
OUTPUT_DIR = '/Net/Groups/BSI/scratch/ppapastefanou/simulations/QPy/25_transient_latin_hypercube_with_std_HAINICH_data_full_2024_rs_work_high_gammastem'
post_dir = os.path.join(OUTPUT_DIR, 'post')
rmse_file = os.path.join(post_dir, 'standard_ranking.csv')
parameter_file = os.path.join(OUTPUT_DIR, 'parameters.csv')

In [4]:
df_parameters = pd.read_csv(parameter_file)
df_parameters

,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,silt,sand,root_scale,slope_leaf_close,g_res
0,2.47655,0,0,54.43476,0.00633,4252.17745,0.00426,3.92496,-2.16936,5.57553,0.33987,0.23113,10.03833,3.02659,5.340000e-06
1,2.42381,1,1,45.62512,0.00638,3545.86803,0.00591,4.02606,-2.12794,4.50922,0.36202,0.17809,37.53206,3.13563,1.378900e-04
2,0.49068,2,2,237.75605,0.00253,4705.11535,0.00901,3.89074,-2.32027,5.34585,0.33168,0.17145,10.10650,2.97384,1.600000e-06
3,0.84297,3,3,87.40472,0.00205,4131.04293,0.00932,4.17278,-2.19916,6.44809,0.35163,0.24973,48.15152,3.97983,2.770000e-06
4,2.36500,4,4,77.58250,0.00492,4658.32053,0.00321,4.16386,-2.03398,3.69119,0.31503,0.19481,20.35905,2.88395,4.300000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,2.06989,8187,8187,41.81198,0.00441,4253.68419,0.00468,4.15555,-2.34563,3.83214,0.31926,0.16196,17.98533,3.89673,1.222400e-04
8188,1.05633,8188,8188,104.40352,0.00580,4484.96488,0.00743,4.03336,-2.33202,5.72633,0.35972,0.17652,13.07747,2.97505,4.110000e-06
8189,1.24251,8189,8189,237.31518,0.00120,4927.79300,0.00565,3.99115,-2.23854,1.99591,0.33625,0.23317,19.27907,2.75059,3.830000e-06
8190,0.87863,8190,8190,201.49168,0.00937,4236.09659,0.00594,4.18559,-2.33956,6.36137,0.32487,0.17396,49.65049,2.68463,3.253600e-04


In [5]:
df_rmse = pd.read_csv(rmse_file)
df_rmse= pd.merge(df_parameters, df_rmse, on='fid')

In [23]:
df_rmse

,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
0,2.47655,0,0,54.43476,0.00633,4252.17745,0.00426,3.92496,-2.16936,5.57553,...,3.02659,5.340000e-06,0.353956,1.636023e+00,3.635982,3.278169,43.883223,46.947214,NaN,-0.011351
1,2.42381,1,1,45.62512,0.00638,3545.86803,0.00591,4.02606,-2.12794,4.50922,...,3.13563,1.378900e-04,0.495419,1.393737e+00,3.677294,3.278003,41.849852,44.351338,0.217847,0.247036
2,0.49068,2,2,237.75605,0.00253,4705.11535,0.00901,3.89074,-2.32027,5.34585,...,2.97384,1.600000e-06,0.599401,2.615192e+12,4.308080,3.812042,43.545409,44.078317,0.309771,-0.043011
3,0.84297,3,3,87.40472,0.00205,4131.04293,0.00932,4.17278,-2.19916,6.44809,...,3.97983,2.770000e-06,0.506821,6.612575e-01,3.800921,3.437250,42.578713,44.620361,0.210582,0.036487
4,2.36500,4,4,77.58250,0.00492,4658.32053,0.00321,4.16386,-2.03398,3.69119,...,2.88395,4.300000e-07,0.535702,6.681946e-01,3.480868,3.319368,41.327453,43.886202,0.215885,0.186538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,2.06989,8187,8187,41.81198,0.00441,4253.68419,0.00468,4.15555,-2.34563,3.83214,...,3.89673,1.222400e-04,0.850096,8.534337e-01,3.519510,3.309519,41.153867,43.633144,0.208149,0.134496
8188,1.05633,8188,8188,104.40352,0.00580,4484.96488,0.00743,4.03336,-2.33202,5.72633,...,2.97505,4.110000e-06,1.037783,1.017165e+00,3.596244,3.306669,41.307986,43.254849,0.245012,0.006107
8189,1.24251,8189,8189,237.31518,0.00120,4927.79300,0.00565,3.99115,-2.23854,1.99591,...,2.75059,3.830000e-06,0.173054,8.097952e-01,3.532276,3.327841,42.323876,44.479385,0.206516,0.356494
8190,0.87863,8190,8190,201.49168,0.00937,4236.09659,0.00594,4.18559,-2.33956,6.36137,...,2.68463,3.253600e-04,0.759850,1.215865e+00,3.627926,3.254348,40.591261,42.398406,0.234818,0.091970


In [24]:
df_rmse.sort_values(by = 'rmse_psi_stem')

,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
427,0.19404,427,427,209.75632,0.00734,4664.70729,0.00444,4.10383,-2.27861,5.22847,...,2.23663,9.400000e-07,0.079935,0.550721,5.311977,4.555662,43.536863,42.711057,0.233405,0.058971
2487,0.15903,2487,2487,152.62774,0.00817,4717.36760,0.00450,4.19559,-2.16217,5.42613,...,3.58401,7.860000e-05,0.080030,0.675412,5.902216,5.166284,44.630844,43.695755,0.208870,0.000161
2691,0.14607,2691,2691,112.87180,0.00761,3325.85438,0.00363,3.86088,-2.24154,5.62780,...,2.04493,6.443000e-05,0.080337,0.708841,5.459325,4.596387,43.192172,42.971093,0.203501,0.047349
7404,1.36522,7404,7404,247.67072,0.00495,4281.23780,0.00252,3.91982,-2.43283,1.97953,...,2.34170,5.984600e-04,0.080635,0.578327,3.603678,3.303096,41.655773,43.650046,0.224217,0.346972
7275,0.16165,7275,7275,181.90802,0.00856,4164.86027,0.00943,3.81579,-2.36399,5.78406,...,3.94089,1.200000e-07,0.082476,0.586137,5.678036,4.974802,43.972674,43.718994,0.219591,0.018682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7835,1.72979,7835,7835,213.05090,0.00066,3308.69617,0.00636,3.90892,-2.20237,6.01900,...,3.24733,8.067000e-04,8.956105,1.636023,8.967080,7.908745,42.631792,43.838896,0.768353,0.000000
2084,0.81229,2084,2084,98.48692,0.00090,4740.55082,0.00691,3.89931,-2.20968,5.37322,...,2.03125,6.205900e-04,9.484179,1.636023,8.967080,7.908745,42.255352,42.848880,0.768353,0.000000
5258,1.10600,5258,5258,200.06650,0.00056,4624.43140,0.00365,4.03210,-2.42984,5.44110,...,2.20376,8.913900e-04,9.714110,1.636023,8.967080,7.908745,42.371482,43.185080,0.768353,0.000000
4796,0.97519,4796,4796,225.69843,0.00118,3909.84677,0.00872,3.84107,-2.48665,5.40645,...,3.37874,5.800500e-04,9.891059,1.636023,8.967080,7.908745,42.243714,42.802240,0.768353,0.000000


In [8]:
df_rmse.sort_values(by = 'rmse_gpp_22')

,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
7754,2.48495,7754,7754,154.87634,0.00634,4987.42996,0.00975,4.16949,-2.27956,3.62986,...,3.81376,3.000000e-07,1.126099,1.297215,3.398995,3.368814,41.693288,44.237323,0.319068,0.098819
2697,2.12647,2697,2697,241.05169,0.00601,4913.37641,0.00682,4.14004,-2.40880,4.55758,...,3.98834,2.118100e-04,1.287892,1.441252,3.409681,3.363691,41.813709,44.366879,0.346693,0.118424
2934,1.77796,2934,2934,238.18448,0.00516,4958.81355,0.00845,3.95408,-2.46965,3.89619,...,3.76183,5.170000e-04,1.167405,1.387978,3.410138,3.353377,41.537288,43.888658,0.340414,0.121393
7017,2.46995,7017,7017,193.49889,0.00979,4984.26229,0.00874,3.80526,-2.40552,5.41507,...,3.96531,1.690000e-06,1.501849,1.392573,3.413447,3.340568,41.910604,44.069158,0.323472,0.073972
761,2.08287,761,761,187.20858,0.00231,4987.26328,0.00499,4.18721,-2.44665,5.15533,...,2.87719,8.179000e-04,1.460965,1.272467,3.414303,3.339994,41.930201,44.606217,0.287261,0.036157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8070,2.22580,8070,8070,118.84675,0.00125,4811.64310,0.00946,4.16010,-2.16394,5.13672,...,2.12608,1.733800e-04,3.919532,1.636023,8.967080,7.908745,42.487351,43.481485,0.768353,0.000000
8177,0.34960,8177,8177,181.74401,0.00393,4861.32974,0.00762,3.91171,-2.33807,5.58283,...,3.17816,9.125500e-04,2.425125,1.636023,8.967080,7.908745,42.206005,42.709030,0.768353,0.000000
6096,1.56522,6096,6096,78.74130,0.00085,3414.81626,0.00748,3.96991,-2.23326,3.21502,...,3.65644,4.917800e-04,3.864933,1.636023,8.967080,7.908745,42.465275,43.424217,0.768353,0.000000
8157,1.33654,8157,8157,40.08015,0.00090,4202.16664,0.00798,4.01310,-2.28648,1.84835,...,2.65951,9.645600e-04,4.853585,1.636023,8.967080,7.908745,42.278215,42.908649,0.768353,0.000000


In [10]:
df_rmse.sort_values(by = 'rmse_le_22')

,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
3149,10.40712,3149,3149,230.96256,0.00920,3374.24589,0.00493,4.00372,-2.02498,5.64916,...,2.00350,0.000046,1.072744,1.364333,3.751290,3.264896,40.321883,41.878681,0.301500,0.149624
10151,14.47056,10151,10151,278.68868,0.00551,4538.46973,0.00252,3.83406,-2.31131,4.53641,...,2.00116,0.001005,1.547428,1.379603,3.504704,3.294622,40.363941,41.964899,0.338284,0.050446
3595,9.06035,3595,3595,319.64285,0.00774,3312.08799,0.00290,4.01402,-2.32983,6.32847,...,2.11963,0.000213,1.610111,1.426058,3.753809,3.275080,40.453171,42.039693,0.330786,0.087355
11999,9.21477,11999,11999,228.54926,0.00347,4830.61072,0.00559,3.82701,-2.01997,6.41833,...,2.21520,0.000032,1.452004,1.367315,3.499364,3.306824,40.480916,41.815034,0.281605,-0.069129
3127,9.97742,3127,3127,340.34340,0.00863,4553.22308,0.00279,3.93079,-2.17020,4.54167,...,2.10944,0.000202,1.385813,1.470797,3.495492,3.296670,40.482066,42.251384,0.364924,0.056674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,11.46134,9214,9214,333.22421,0.00461,4908.31800,0.00947,4.19369,-2.39339,2.00080,...,3.65762,0.000131,0.136086,0.720983,3.527768,3.443279,44.572249,47.390466,0.229644,0.452535
9909,10.70849,9909,9909,257.23348,0.00497,4892.65831,0.00762,4.19619,-2.19791,2.22920,...,3.91544,0.000001,0.181104,0.817960,3.528456,3.441383,44.576922,47.445450,0.230885,0.442355
10327,13.25033,10327,10327,294.17533,0.00630,4755.31318,0.00809,4.15097,-2.35562,1.54595,...,3.67130,0.000006,0.236037,0.902521,3.543839,3.426823,44.588781,47.417947,0.235074,0.440006
8715,10.64554,8715,8715,341.85760,0.00416,4690.18072,0.00531,4.15844,-2.45572,1.70101,...,3.96585,0.000025,0.143389,0.663987,3.549072,3.421903,44.628519,47.542008,0.233002,0.430717


In [11]:
df_rmse.sort_values(by = 'rmse_sapflow_norm')

,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
8769,14.85243,8769,8769,202.74905,0.00494,4886.28348,0.00550,3.89629,-2.20128,3.02352,...,2.07386,0.001862,0.216624,0.615854,3.493144,3.381827,42.169513,44.530493,0.212385,0.324942
5453,11.26793,5453,5453,227.76831,0.00987,4421.87688,0.00299,3.95450,-2.12537,3.16784,...,2.96497,0.001182,0.172018,0.684824,3.546334,3.369981,43.016522,45.708919,0.213865,0.367815
11566,10.67368,11566,11566,227.69345,0.00451,4787.87503,0.00935,3.99569,-2.25988,3.09835,...,2.62730,0.000837,0.162049,0.719270,3.502672,3.404033,43.720689,46.527891,0.214991,0.357172
3848,8.87977,3848,3848,224.61761,0.00402,4551.89275,0.00416,3.82674,-2.25540,2.74857,...,2.05278,0.001120,0.106947,0.612935,3.557333,3.353946,42.258772,44.388811,0.215738,0.360348
12797,10.72626,12797,12797,218.20974,0.00656,4189.06641,0.00749,4.14180,-2.28338,3.06687,...,2.66035,0.001423,0.205925,0.669524,3.569869,3.359732,43.480379,46.394111,0.215928,0.356638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13951,8.38614,13951,13951,302.25878,0.00903,4990.41001,0.00981,3.92260,-2.20298,5.93936,...,2.35943,0.000478,3.691619,1.636023,8.967080,7.908745,42.612189,43.788963,0.768353,0.000000
78,10.87149,78,78,308.27290,0.00165,4650.64443,0.00636,4.00208,-2.17458,3.85522,...,2.01512,0.000815,4.109784,1.636023,8.301800,7.908745,42.668943,44.379717,0.768353,0.000000
9628,10.67846,9628,9628,301.01999,0.00332,4531.91330,0.00681,4.05337,-2.48974,6.45671,...,2.59745,0.001090,3.769035,1.636023,8.967080,7.908745,42.382901,43.221816,0.768353,0.000000
9611,13.53405,9611,9611,238.47990,0.00145,4282.20355,0.00749,3.95898,-2.45348,2.40205,...,2.60739,0.000922,4.629682,1.636023,8.303536,7.908745,42.309084,43.663675,0.768353,0.000000


In [12]:
df_rmse.sort_values(by = 'slope_gpp17-19').head(10)
df_rmse.mean()

k_xylem_sat             11.250002
id                    8191.500000
fid                   8191.500000
kappa_stem             274.999992
kappa_leaf               0.005250
k_latosa              4150.000046
g0                       0.006250
g1                       4.000000
psi50_close             -2.250000
root_dist                3.949999
silt                     0.330000
sand                     0.205000
root_scale              24.853390
slope_leaf_close         3.000000
g_res                    0.000263
rmse_psi_stem            1.025237
rmse_psi_stem_norm       1.189261
rmse_gpp_22              3.765622
rmse_gpp_24              3.491272
rmse_le_22              42.282890
rmse_le_24              44.616817
rmse_sapflow_norm        0.318778
slope_gpp17-19           0.227897
dtype: float64

In [10]:
#slice_psi_stem = df_rmse[df_rmse['rmse_psi_stem'] < 0.12]
df_rmse.iloc[[7816, 729, 1873]]

,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
7816,8.32894,7816,7816,479.42230,0.00384,3793.04989,0.02819,4.04708,-2.32070,3.61470,...,3.49947,0.00001,0.254685,1.267467,3.621151,3.331374,43.093831,45.183578,0.357550,0.344950
729,9.19161,729,729,151.88494,0.00189,4016.03318,0.01239,4.04749,-2.38129,2.66727,...,1.92751,0.00000,0.179592,1.099305,3.648614,3.332639,43.441201,45.734432,0.232563,0.405473
1873,7.03762,1873,1873,239.14073,0.00279,4081.71164,0.03000,3.53622,-2.24308,3.83950,...,2.85748,0.00000,0.372116,1.081454,3.573063,3.311491,43.055917,44.770392,0.303777,0.254228


In [11]:
slice_psi_stem = df_rmse[df_rmse['rmse_le_24'] < 46.9]
slice_psi_stem.shape

(15660, 23)

In [12]:
slice_psi_stem = df_rmse[df_rmse['rmse_le_22'] < 43.88]
slice_psi_stem.shape

(14890, 23)

In [13]:
slice_psi_stem = df_rmse[df_rmse['rmse_gpp_22'] < 3.63]
slice_psi_stem.shape

(6635, 23)

In [14]:
slice_psi_stem = df_rmse[df_rmse['rmse_gpp_24'] < 3.27]
slice_psi_stem.shape

(1387, 23)

In [15]:
slice_psi_stem = df_rmse[(df_rmse['rmse_psi_stem'] < 0.12) & (df_rmse['rmse_gpp_24'] < 3.27)]
slice_psi_stem.shape

(102, 23)

In [25]:
slice_psi_stem = df_rmse[(df_rmse['rmse_psi_stem'] < 0.12) & (df_rmse['rmse_gpp_22'] < 3.63)]
slice_psi_stem.shape

(171, 23)

In [26]:
slice_psi_stem = df_rmse[(df_rmse['rmse_psi_stem'] < 0.11) & (df_rmse['rmse_gpp_22'] < 3.63)& (df_rmse['rmse_le_22'] < 43.88)]
slice_psi_stem.shape


(119, 23)

In [27]:
slice_psi_stem = df_rmse[(df_rmse['rmse_psi_stem'] < 0.09) & (df_rmse['rmse_gpp_22'] < 3.63)& (df_rmse['rmse_le_22'] < 43.88)]
slice_psi_stem.shape
slice_psi_stem


,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
79,1.63758,79,79,224.11674,0.00916,4199.57379,0.00653,3.91854,-2.48263,2.55689,...,3.78211,9.706000e-05,0.087809,0.660564,3.599020,3.348743,42.753789,45.002759,0.224086,0.393500
245,1.99946,245,245,213.29633,0.00258,4226.82224,0.00687,4.08717,-2.37336,1.96981,...,3.89291,4.122000e-05,0.087781,0.599083,3.594734,3.363071,43.325066,45.833418,0.227327,0.399817
2127,1.95636,2127,2127,214.28164,0.00392,4519.57683,0.00657,3.87785,-2.22646,2.38777,...,2.85168,6.900000e-07,0.088758,0.634101,3.556501,3.345580,42.897672,45.269588,0.222457,0.357662
2994,1.72598,2994,2994,245.10174,0.00799,4495.27442,0.00892,4.05964,-2.49794,2.45595,...,2.53002,8.425800e-04,0.086457,0.647993,3.548618,3.356473,42.570736,44.912221,0.217131,0.357612
3042,1.33597,3042,3042,232.82441,0.00898,4259.34892,0.00858,4.00215,-2.33194,2.52729,...,2.54645,9.324000e-05,0.087092,0.684080,3.594906,3.302358,42.234504,44.439298,0.221836,0.333766
3262,1.95705,3262,3262,247.17364,0.00423,4113.73654,0.00275,4.12134,-2.04202,2.38079,...,2.94618,9.836900e-04,0.087979,0.595161,3.603055,3.313759,42.861627,45.601144,0.218520,0.335542
6701,1.61662,6701,6701,247.72920,0.00865,4975.23267,0.00836,4.02557,-2.32313,1.73888,...,3.47753,2.809000e-05,0.083240,0.592973,3.503625,3.398284,42.485817,44.682141,0.242663,0.367164
6913,1.76593,6913,6913,233.71888,0.00155,4902.25259,0.00750,4.16707,-2.23601,2.09019,...,2.38713,7.439100e-04,0.087824,0.617794,3.514041,3.355666,42.244296,44.539069,0.212730,0.332440
7014,1.80955,7014,7014,225.11877,0.00219,4596.21841,0.00730,3.95030,-2.22104,2.06245,...,3.92445,4.660000e-06,0.088803,0.621255,3.545312,3.375157,43.215779,45.673481,0.223798,0.381669
7404,1.36522,7404,7404,247.67072,0.00495,4281.23780,0.00252,3.91982,-2.43283,1.97953,...,2.34170,5.984600e-04,0.080635,0.578327,3.603678,3.303096,41.655773,43.650046,0.224217,0.346972


In [68]:
slice_psi_stem[['fid', 'rmse_psi_stem','k_xylem_sat', 'kappa_stem', 'kappa_leaf', 'root_dist']].sort_values(by = 'root_dist', ascending=False)['fid'].to_list()

[8635,
 2739,
 13454,
 11833,
 6415,
 14475,
 12545,
 4272,
 2834,
 12739,
 15962,
 14426,
 11843,
 7251,
 7912,
 5702,
 16294,
 15671,
 13467,
 1723,
 5113,
 15723,
 8247,
 4693,
 9946,
 5275,
 2877,
 11590,
 2120,
 11447,
 9703,
 1663,
 8269,
 2281,
 10532,
 10033,
 4914,
 2214,
 9769,
 851,
 4972,
 380,
 6129,
 8025,
 5962,
 7720,
 8949,
 6007]

In [21]:
slice_psi_stem = df_rmse[(df_rmse['slope_gpp17-19'] < 0.0) & (df_rmse['rmse_psi_stem'] < 0.2)]
slice_psi_stem


,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
381,0.12240,381,381,162.14703,0.00482,4260.33609,0.00951,3.94476,-2.20783,4.61990,...,2.87146,0.000344,0.134257,0.630689,6.004869,5.227209,44.127955,43.239769,0.221561,-0.089135
4045,0.10339,4045,4045,45.00756,0.00476,4654.27923,0.00737,3.92975,-2.38410,6.38677,...,2.45349,0.000078,0.116729,1.235670,6.116937,5.322672,44.463338,43.161367,0.204470,-0.087471
4066,0.10821,4066,4066,66.14147,0.00622,4672.00869,0.00845,4.01128,-2.10706,3.68927,...,3.36370,0.000076,0.198918,0.875486,6.379480,5.579642,43.644540,42.088282,0.178833,-0.015237
5885,0.15973,5885,5885,242.98024,0.00433,4861.06421,0.00868,4.01426,-2.15191,6.19720,...,2.34042,0.000001,0.123952,0.680939,5.672035,4.897720,44.743167,43.945571,0.237936,-0.042985
6007,0.10989,6007,6007,65.04710,0.00653,4938.24992,0.00710,3.88250,-2.13594,5.88485,...,3.55649,0.000734,0.124515,1.476168,6.388950,5.604265,43.924809,42.408724,0.181282,-0.020307
7559,0.13375,7559,7559,175.61356,0.00472,4168.13213,0.00882,4.17892,-2.26537,4.34034,...,2.21167,0.000025,0.085654,0.593257,5.770397,5.019284,44.234832,43.052423,0.245008,-0.025432


In [18]:
slice_psi_stem = df_rmse[(df_rmse['slope_gpp17-19'] < -0.1) &(df_rmse['slope_gpp17-19'] > -0.5)]
print(slice_psi_stem.shape)
y = slice_psi_stem.mean().to_frame()
slice_psi_stem.sort_values(by = 'slope_gpp17-19')['fid'].to_list()
slice_psi_stem.sort_values(by = 'slope_gpp17-19')

(20, 23)


,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
2080,1.77616,2080,2080,165.79634,0.00229,4911.41677,0.00708,3.88986,-2.16329,6.48300,...,3.75316,2.893000e-05,1.408967,1.305547e+00,3.558127,3.374661,40.753927,41.853727,0.272112,-0.167691
575,1.33845,575,575,246.37940,0.00184,4889.66628,0.00913,3.95555,-2.49233,6.39784,...,2.57823,1.170000e-06,1.682502,1.412390e+00,3.553567,3.333644,40.565101,41.594371,0.293438,-0.163879
5145,1.90842,5145,5145,113.61040,0.00195,4948.67694,0.00614,4.10169,-2.47483,6.43009,...,2.16339,4.000000e-07,1.717919,1.224271e+00,3.545855,3.332828,40.482762,41.812004,0.268875,-0.153952
7369,1.34436,7369,7369,179.62064,0.00446,4881.85707,0.00557,3.94751,-2.14958,6.09557,...,3.50636,2.566000e-05,1.279426,1.304566e+00,3.623376,3.392312,40.735537,41.710740,0.273288,-0.152551
7618,1.08870,7618,7618,219.72689,0.00713,4887.23882,0.00712,4.16263,-2.47597,6.00900,...,3.14734,5.551000e-05,1.523950,1.393703e+00,3.579607,3.370896,40.766010,42.107387,0.279142,-0.151465
7247,1.56641,7247,7247,86.89328,0.00207,4931.72700,0.00865,4.03552,-2.18064,6.38357,...,3.61870,6.900000e-07,1.308454,1.041356e+00,3.569018,3.386549,41.075112,42.551718,0.256942,-0.149579
5924,1.14334,5924,5924,82.76617,0.00167,4608.57612,0.00425,4.09333,-2.15764,6.49678,...,3.86618,6.157000e-05,1.140691,8.657401e-01,3.800625,3.515688,41.815864,43.014573,0.255320,-0.142303
3863,1.96077,3863,3863,201.18513,0.00748,4372.28094,0.00515,3.96569,-2.08633,6.47992,...,3.39213,9.592800e-04,1.345200,1.119978e+00,3.660883,3.352371,41.262588,42.618099,0.262753,-0.134553
832,0.73351,832,832,83.56912,0.00218,3576.08465,0.00993,4.12662,-2.13209,6.34224,...,3.65061,9.300000e-07,0.962129,7.886467e-01,4.171743,3.593403,42.025502,42.778441,0.242494,-0.131842
174,1.49422,174,174,117.09251,0.00170,4413.60882,0.00973,4.01529,-2.36708,6.36326,...,3.94786,3.544000e-05,1.501618,1.200449e+00,3.606146,3.373485,41.639166,43.358038,0.261549,-0.128373


In [28]:
slice_psi_stem = df_rmse[(df_rmse['rmse_psi_stem'] < 0.09) & (df_rmse['rmse_gpp_22'] < 3.63)]
print(slice_psi_stem.shape)
x = slice_psi_stem.mean().to_frame()

(10, 23)


In [31]:
pd.concat([x,y], axis = 1)

,0,0
k_xylem_sat,1.716972,1.517776e+00
id,3978.100000,4.066200e+03
fid,3978.100000,4.066200e+03
kappa_stem,233.103207,1.649017e+02
kappa_leaf,0.005420,3.234000e-03
k_latosa,4456.927421,4.569027e+03
g0,0.006590,7.604500e-03
g1,4.012945,4.040172e+00
psi50_close,-2.316736,-2.250829e+00
root_dist,2.214955,6.213268e+00


In [55]:
slice_psi_stem = df_rmse[(df_rmse['rmse_psi_stem'] < 0.12) & (df_rmse['rmse_gpp_24'] < 3.27)& (df_rmse['rmse_le_24'] < 46.9)& (df_rmse['kappa_stem'] < 146.9)]
print(slice_psi_stem.shape)
slice_psi_stem

(7, 21)


,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,sand,root_scale,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
570,5.58715,570,570,138.93311,0.00323,3615.86920,0.01950,3.79973,-2.36412,2.45911,...,0.17076,16.44481,0.112196,0.616100,3.827322,3.266784,41.828141,43.124194,0.224596,0.321514
2530,9.45306,2530,2530,111.68811,0.00279,3712.84509,0.01964,3.65794,-2.31822,3.54702,...,0.16857,67.02622,0.113407,0.862513,3.846772,3.259248,41.175883,41.884090,0.226077,0.317868
6412,5.46814,6412,6412,146.51404,0.00260,4133.81270,0.01169,3.89282,-2.02338,3.32081,...,0.22142,34.51727,0.104739,0.682451,3.745053,3.251664,41.756178,43.256754,0.219417,0.299623
6867,8.62239,6867,6867,122.56815,0.00944,3937.56285,0.01185,3.70403,-2.25629,2.02419,...,0.20436,11.08581,0.111987,0.967265,3.806384,3.255782,41.515359,42.665564,0.225196,0.328176
9026,9.47338,9026,9026,143.72503,0.01006,3860.43463,0.02530,3.54020,-2.11238,3.28573,...,0.22127,46.03300,0.098497,0.676222,3.800262,3.254857,42.282697,43.280805,0.226166,0.313970
11837,8.39565,11837,11837,126.49880,0.01950,3972.18215,0.01995,4.10244,-2.11874,3.34654,...,0.24502,60.57907,0.110067,0.801875,3.736705,3.266921,42.296364,43.945379,0.224091,0.306577
15834,4.57136,15834,15834,118.23623,0.00432,3568.28870,0.01491,3.74675,-2.31776,2.81838,...,0.19029,21.83065,0.111453,0.854540,3.830005,3.267793,42.435157,44.170979,0.225180,0.335837


In [30]:
slice_psi_stem.describe()

,k_xylem_sat,id,fid,kappa_stem,kappa_leaf,k_latosa,g0,g1,psi50_close,root_dist,...,slope_leaf_close,g_res,rmse_psi_stem,rmse_psi_stem_norm,rmse_gpp_22,rmse_gpp_24,rmse_le_22,rmse_le_24,rmse_sapflow_norm,slope_gpp17-19
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,1.716972,3978.100000,3978.100000,233.103207,0.005420,4456.927421,0.006590,4.012945,-2.316736,2.214955,...,3.068016,3.433600e-04,0.086638,0.623133,3.566349,3.346219,42.624506,44.960316,0.223477,0.360614
std,0.233585,2826.549229,2826.549229,13.579406,0.003001,297.884323,0.002242,0.096165,0.140081,0.281052,...,0.641683,3.987212e-04,0.002643,0.033606,0.037938,0.031394,0.499464,0.668700,0.007944,0.024612
min,1.335970,79.000000,79.000000,213.296330,0.001550,4113.736540,0.002520,3.877850,-2.497940,1.738880,...,2.341700,6.900000e-07,0.080635,0.578327,3.503625,3.302358,41.655773,43.650046,0.212730,0.332440
25%,1.621860,2343.750000,2343.750000,224.367247,0.002915,4234.953910,0.006540,3.927440,-2.417962,2.000260,...,2.534127,3.137250e-05,0.086616,0.596141,3.546138,3.321715,42.304677,44.574837,0.219349,0.338399
50%,1.745955,3152.000000,3152.000000,233.271645,0.004590,4388.256110,0.007085,4.013860,-2.327535,2.235490,...,2.898930,9.515000e-05,0.087795,0.619524,3.575618,3.352204,42.662263,44.957490,0.223128,0.357637
75%,1.919658,6860.000000,6860.000000,246.655665,0.008485,4577.058015,0.008145,4.080288,-2.228847,2.438905,...,3.705965,7.075475e-04,0.087940,0.644520,3.597992,3.361421,42.888661,45.518255,0.224184,0.378043
max,1.999460,7404.000000,7404.000000,247.729200,0.009160,4975.232670,0.008920,4.167070,-2.042020,2.556890,...,3.924450,9.836900e-04,0.088803,0.684080,3.603678,3.398284,43.325066,45.833418,0.242663,0.399817


In [35]:
slice_psi_stem.sort_values(by = 'rmse_psi_stem', ascending=False)['fid'].to_list()

[4500,
 627,
 12283,
 3609,
 15405,
 3668,
 7024,
 1885,
 4878,
 8000,
 13183,
 9586,
 11197,
 6050,
 7527,
 11823,
 13293,
 5121,
 10434,
 11336,
 157,
 6801,
 5289,
 15197,
 16348,
 15201,
 7086,
 4310,
 5195,
 8556,
 9535,
 15691,
 1632,
 14985,
 1556,
 10872,
 2581,
 4763,
 1194,
 14746,
 11448,
 15062,
 14805,
 4970,
 12127,
 2579,
 3058,
 2210,
 11083,
 13774,
 12171,
 5757,
 632,
 3664,
 1770,
 1094,
 2322,
 9941,
 9643,
 2475,
 8665,
 7131,
 5476,
 11688,
 6101,
 11897,
 14544,
 1862,
 6982,
 3470,
 7735,
 466,
 9897,
 5026,
 2531,
 8284,
 3194,
 6816,
 5145,
 6998,
 1458,
 11399,
 10628,
 13935,
 2736,
 4622,
 7427,
 15648,
 4224,
 8285,
 15961,
 1311,
 12294,
 10543,
 8524,
 346,
 4703,
 4116,
 11248,
 3196,
 11937,
 14464,
 6435,
 10906,
 11288,
 397,
 7708,
 15574,
 15256,
 12537,
 6817,
 11199,
 3405,
 11513,
 4318,
 1423,
 9395,
 13076,
 2883,
 6315,
 9187,
 2927,
 631,
 10294,
 15494,
 14704,
 10128,
 9362,
 15974,
 7260,
 2587,
 4761,
 555,
 7923,
 2098,
 815,
 5826,
 1

In [40]:
slice_psi_stem = df_rmse[(df_rmse['slope_gpp17-19'] < -0.1) &(df_rmse['slope_gpp17-19'] > -0.5)]
slice_psi_stem.sort_values(by = 'slope_gpp17-19', ascending=False)['fid'].to_list()


[8193,
 454,
 11671,
 6521,
 12277,
 4576,
 9792,
 1387,
 13066,
 10347,
 12123,
 4217,
 15260,
 7585,
 10066,
 8354,
 294,
 10220,
 14573,
 12272,
 6477,
 1730,
 7831,
 7424,
 5047,
 5024,
 15688,
 5637,
 7558,
 1084,
 10299,
 12937,
 8971,
 2968,
 9942,
 6953,
 9210,
 6655,
 10759,
 11797,
 8514,
 1662,
 3815,
 9177,
 11373,
 12779,
 14038,
 4359,
 14222,
 8408,
 6225,
 1101,
 5157,
 12958,
 10141,
 2392,
 1997,
 16113,
 13579,
 1133,
 13083,
 5652,
 5243,
 4493,
 11157,
 10457,
 10920,
 2202,
 6333,
 4173,
 7428,
 16250,
 9331,
 5980,
 14936,
 16171,
 12440,
 15008,
 14335,
 3499,
 69,
 13060,
 4508,
 9788,
 6851,
 6827,
 12201,
 14659,
 14243,
 11431,
 7287,
 7444,
 2867,
 12635,
 12308,
 1222,
 7394,
 10562,
 12908,
 7520,
 2707,
 8337,
 15823,
 14815,
 5298,
 13406,
 13861,
 4897,
 4536,
 2139,
 13751,
 15676,
 8547,
 12258,
 7126,
 9644,
 16367,
 4738,
 7064,
 14467,
 15529,
 12798,
 1252,
 6021,
 3641,
 4403,
 1816,
 14296,
 6126,
 768,
 9512,
 4955,
 2096,
 5841,
 935,
 12343,

KeyError: 'slope_gpp17'